# **_Living Learning Community Flowrate Data Visualization Tool_**
## For: CEE 6110 - Hydroinformatics Semester Project <br>
>### Developed by Joseph Brewer and Jaxon White <br>
>> This script was developed as part of a semester project that tests a <br>
>> hydroinformatics data architecture for high-resolution smart meter water-use <br>
>> in a case study of multi-resident structures.  The script is a basic <br>
>> visualization tool for the researcher to quickly, and simply, query collected<br>
>> one-second resolution hot and cold water flowrate data. <br> 
 
### _Overview:_  
> 1. The script connects to a remote InfluxDB time-series database.  <br>
> 2. Queries the stored one-second resolution flowrate data <br>
> 3. Resamples the queried data to the desired resolution <br>
> 4. Visualizes the results <br>

**Required python libraries** <br>
 - pandas <br>
 - influxdb <br>
 - matplotlib.pyplot <br>
 - numpy <br>


In [3]:
#import libraries 
%matplotlib notebook

import pandas as pd
from influxdb import InfluxDBClient
import matplotlib.pyplot as plt
import numpy as np

ImportError: Failed to import any qt binding

### _Input Parameters_ <br>
Available dates - 10/10/2018 - 11/10/2018 <br>
Available buildings - C, D, E, F <br>
> Dates - Do not remove 'T' of 'Z' - required influxDB syntax <br>
> bldgID - Do not remove " " or ' ' - required influxDB syntax <br>


In [ ]:
beginDate = "'2018-10-10T05:00:00Z'"
endDate = "'2018-10-10T17:00:00Z'"
bldgID = "'F'"

In [ ]:
# Create client object with InfluxDBClient library
# Set database.
client = InfluxDBClient(host='influxdbubuntu.bluezone.usu.edu', port=8086)
client.switch_database('LLC_FlowData')

In [ ]:
# Build query by concatenating inputs into query.  InfluxDB query language has several
# requirements.  Fields/Tags must be bracketed with " " and the field/tag values must be
# bracketed with ' '.
# Query returns a 'ResultSet" type.  Have to convert to pandas dataframe.
query = """SELECT "flowrate", "source" FROM "flow" WHERE "buildingID" ="""+bldgID+""" AND time >= """+beginDate+""" AND time <= """+endDate+""""""

In [ ]:
# Convert returned ResultSet to Pandas dataframe with list
# and get_points.
# Set dataframe index as datetime.
main = client.query(query)
main_Ls = list(main.get_points(measurement='flow'))
main_Df = pd.DataFrame(main_Ls)
main_Df.sort_values(by=['source'], inplace=True)
main_Df['time'] = pd.to_datetime(main_Df['time'])
main_Df.set_index('time', inplace=True)

In [ ]:
cold_Df = main_Df[main_Df['source'] == 'coldIN'].copy()
cold_Df.sort_index(inplace=True)
cold_Df.drop(['source'], axis ='columns', inplace=True)

hot_Df = main_Df[main_Df['source'] == 'hotIN'].copy()
hot_Df.sort_index(inplace=True)
hot_Df.drop(['source'], axis ='columns', inplace=True)


### _Resample Tool_ <br>
Input resample rule: <br>
>Options are: (X is integer, i.e. 15T = 15 minute resample)
>>Daily, XD <br>
>>Weekly, XW <br>
>>Hourly, XH <br>
>>Minute, XT <br>


In [ ]:

resampleRule = '15T'

hotFinalAvg = hot_Df.resample(rule=resampleRule, base=0).mean()
#hotFinalMax = hot_Df.resample(rule=resampleRule, base=0).max()
#hotFinalMin = hot_Df.resample(rule=resampleRule, base=0).min()
hotFinalSum = hot_Df.resample(rule=resampleRule, base=0).sum()

coldFinal = cold_Df.resample(rule = resampleRule , base=0).mean()
#coldFinalMax = cold_Df.resample(rule=resampleRule, base=0).max()
#coldFinalMin = cold_Df.resample(rule=resampleRule, base=0).min()
coldFinalSum = cold_Df.resample(rule=resampleRule, base=0).sum()


In [ ]:
# Plot results - raw data, resampled data, resampled with stats.
# Initialize figures and subplots
gridsize=(3,2)
fig=plt.figure(figsize=(10,6))
fig.autofmt_xdate()
fig.suptitle('Water Use Data Vis/Analysis for Building: '+bldgID+' FOR: '+beginDate+'-'+endDate, fontsize=12, weight='bold')

# 1st row - raw data
axHot1 = plt.subplot2grid(gridsize, (0,0))
plt.xticks(fontsize=8, rotation=35)
axHot1.plot(hot_Df, color='red', label='1-Sec HOT Data')
axHot1.set_title('Raw hot water flowrate', fontsize=10, weight ='bold')
axHot1.legend(loc='upper left')
axHot1.set_ylabel('GPM')
axHot1.set_xlim(beginDate, endDate)
axHot1.grid(True)


axCold1 = plt.subplot2grid(gridsize, (0,1))
plt.xticks(fontsize=8, rotation=35)
axCold1.plot(cold_Df, color='blue', label='1-Sec COLD Data')
axCold1.set_title('Raw cold water flowrate', fontsize=10, weight ='bold')
axCold1.legend(loc='upper left')
axCold1.set_ylabel('GPM')
axCold1.set_xlim(beginDate, endDate)
axCold1.grid(True)


# 2nd row - Resampled instant gpm
axHot2 = plt.subplot2grid(gridsize, (1,0))
plt.xticks(fontsize=8, rotation=35)
axHot2.plot(hot_Df, color='grey', alpha=0.5, linewidth=0.5)
axHot2.plot(hotFinalAvg, color='red', label=resampleRule+' resample')
#axHot2.plot(hotFinalMax, color='lightsalmon', label=resampleRule+' max.')
#axHot2.plot(hotFinalMin, color='maroon',label=resampleRule+' min.')
axHot2.set_title('Resampled hot water flowrate', fontsize=10, weight='bold')
hotLegend = axHot2.legend(loc='upper left')
for label in hotLegend.get_texts():
    label.set_fontsize(6)
axHot2.set_ylabel('GPM')
axHot2.set_xlim(beginDate, endDate)
axHot2.grid(True)

axCold2 = plt.subplot2grid(gridsize, (1,1))
plt.xticks(fontsize=8, rotation=35)
axCold2.plot(cold_Df, color='grey', alpha=0.5, linewidth=0.5)
axCold2.plot(coldFinal, color='blue', label=resampleRule+' resample')
#axCold2.plot(coldFinalMax, color='cyan',label=resampleRule+' max.')
#axCold2.plot(coldFinalMin, color='navy',label=resampleRule+' min.')
axCold2.set_title('Resampled cold water flowrate', fontsize=10, weight='bold')
coldLegend = axCold2.legend(loc='upper left')
for label in coldLegend.get_texts():
    label.set_fontsize(6)
axCold2.set_ylabel('GPM')
axCold2.set_xlim(beginDate, endDate)
axCold2.grid(True)


# 3rd row - Resampled volume OR histogram 
axHot3 = plt.subplot2grid(gridsize, (2,0))
#plt.xticks(fontsize=8, rotation=35)
#axHot3.plot(hotFinalSum, color = 'red')
#axHot3.set_title('Total resampled hot water volume', fontsize=10, weight='bold')
#axHot3.set_ylabel('Gallons')
#axHot3.set_xlim(beginDate, endDate)
#axHot3.grid(True)
plt.xticks(fontsize=8)
dataHot = hotFinalAvg['flowrate']
binwidthHot = (dataHot.max()-dataHot.min())/15
axHot3.hist(dataHot, bins=np.arange(min(dataHot), max(dataHot), binwidthHot), color='red')
axHot3.set_xlabel('Gallons per Minute')
axHot3.set_ylabel('Freq.')
axHot3.set_title('Resampled GPM distribution', fontsize=10, weight='bold')


axCold3 = plt.subplot2grid(gridsize, (2,1))
#plt.xticks(fontsize=8, rotation=35)
#axCold3.plot(coldFinalSum, color='blue')
#axCold3.set_title('Total resampled hot water volume', fontsize=10, weight='bold')
#axCold3.set_ylabel('Gallons')
#axCold3.set_xlim(beginDate, endDate)
#axCold3.grid(True)
plt.xticks(fontsize=8)
dataCold = coldFinal['flowrate']
binwidthCold = (dataCold.max()-dataCold.min())/15
axCold3.hist(dataCold, bins=np.arange(min(dataCold), max(dataHot), binwidthCold), color='blue')
axCold3.set_xlabel('Gallons per Minute')
axCold3.set_ylabel('Freq.')
axCold3.set_title('Resampled GPM distribution', fontsize=10, weight='bold')

plt.tight_layout(pad=5, w_pad=2, h_pad=2.5)
plt.show()